In [ ]:
{'ADCM_RD': 'AS',
 'ADCM_JobType': 'STP', #wbs3
 'ADCM_DIN': '342', # wbs3_id
 'ADCM_Level': 'L01', # wbs2
 'ADCM_Title': '01p', # wbs1
 'id': 849}

In [1]:
import os

import ifcopenshell
from ifcopenshell.util.element import get_psets

In [2]:
from src.ifc_to_nx_converter import IfcToNxConverter
node_attributes = IfcToNxConverter.node_attributes

In [7]:
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\IFC_test\\notebooks'

In [3]:
model = ifcopenshell.open('../Amundsena_IFC_24/DOU_510_KJ1.ifc')

In [8]:
{i.is_a() for i in model.by_type("IfcTypeObject")}

{'IfcBuildingElementProxyType',
 'IfcColumnType',
 'IfcSlabType',
 'IfcStairType',
 'IfcWallType'}

In [9]:
building = model.by_type("IfcBuilding")[0]

In [13]:
type(building.id())

int

In [4]:
def _filter_ifc(element) -> bool:
    return (
            element.is_a("IfcElement")
            or element.is_a("IfcSpatialStructureElement")
            or element.is_a("IfcObjectDefinition")
    ) and not (element.is_a("IfcGrid") or element.is_a("IfcAnnotation"))

In [6]:
def get_all_children(element):
    all_children = set()

    # Используем связь element.ContainsElements
    # if hasattr(element, "ContainsElements"):
    if element.is_a('IfcSpatialStructureElement'):
        for rel in element.ContainsElements:
            if rel.is_a("IfcRelContainedInSpatialStructure"):
                all_children.update(rel.RelatedElements)

    # Используем связь element.IsDecomposedBy
    # if hasattr(element, "IsDecomposedBy"):
    if element.is_a('IfcObjectDefinition'):
        for rel in element.IsDecomposedBy:
            if rel.is_a("IfcRelAggregates"):
                all_children.update(rel.RelatedObjects)

    return list(filter(lambda el: _filter_ifc(el), all_children))

In [10]:
children = get_all_children(building)

In [12]:
for i in children:
    print(node_attributes(i)["name"])

Площадка 1-2
Площадка 2-3
низ металл
Плита Пм 2 на отм. +7.100. Опалубка
Этаж 1


In [32]:
stor = children[0]
for i in get_all_children(stor):
    print(node_attributes(i)["name"])

Перекрытие:КЖ0_Перекрытие:430323
Базовая стена:Монолит-200 1 эт.:441463
Базовая стена:Монолит-200 1 эт.:441467
Базовая стена:Монолит-200 1 эт.:441471
Базовая стена:Монолит-200 1 эт.:441475
Базовая стена:Монолит-200 1 эт.:441484
Базовая стена:Монолит-200 1 эт.:441488
Базовая стена:Монолит-200 1 эт.:441492
Базовая стена:Монолит-200 1 эт.:441496
Базовая стена:Монолит-200 1 эт.:441500
Базовая стена:Монолит-200 1 эт.:441504
Базовая стена:Монолит-200 1 эт.:441508
Базовая стена:Монолит-200 1 эт.:441512
Базовая стена:Монолит-200 1 эт.:441516
Базовая стена:Монолит-200 1 эт.:441550
Базовая стена:Монолит-200 1 эт.:441554
Базовая стена:Монолит-200 1 эт.:441563
Базовая стена:Монолит-200 1 эт.:441567
Базовая стена:Монолит-200 1 эт.:441571
Базовая стена:Монолит-250 1 этаж:441579
Базовая стена:Монолит-200 1 эт.:441583
Базовая стена:Монолит-250 1 этаж:441587
Базовая стена:Монолит-200 1 эт.:441591
Базовая стена:Монолит-200 1 эт.:441595
Базовая стена:Монолит-200 1 эт.:441599
Базовая стена:Монолит-200 1 э

In [35]:
psets = ifcopenshell.util.element.get_psets(building, psets_only=True)
psets

{'Pset_BuildingCommon': {'Reference': 'Сведения о проекте',
  'NumberOfStoreys': 12,
  'IsLandmarked': 'UNKNOWN',
  'id': 23439},
 'Pset_BuildingElementProxyCommon': {'Reference': 'Сведения о проекте',
  'IsExternal': False,
  'id': 23440},
 'Pset_BuildingStoreyCommon': {'Reference': 'Сведения о проекте',
  'AboveGround': 'UNKNOWN',
  'id': 23441},
 'Pset_BuildingSystemCommon': {'Reference': 'Сведения о проекте', 'id': 23442}}

In [34]:
psets = ifcopenshell.util.element.get_psets(stor, psets_only=True)
psets.keys()

dict_keys(['Pset_BuildingStoreyCommon'])

In [36]:
elem = get_all_children(stor)[0]
psets = ifcopenshell.util.element.get_psets(elem, psets_only=True)
psets.keys()

dict_keys(['Pset_SlabCommon', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfSlab', 'ADCM'])

In [37]:
psets["ADCM"]

{'ADCM_RD': 'КЖ1',
 'ADCM_GESN': '06-01-041-01',
 'ADCM_Level': 'L01',
 'ADCM_Part': '1',
 'ADCM_Title': 'АИ-11 О/22',
 'id': 243}

In [38]:
elem.id(), elem.Name

(227, 'Перекрытие:КЖ0_Перекрытие:430323')

In [39]:
psets

{'Pset_SlabCommon': {'IsExternal': False,
  'ThermalTransmittance': 4.754545454545455,
  'id': 237,
  'Reference': 'Перекрытие:КЖ0_Перекрытие',
  'PitchAngle': 0.0,
  'LoadBearing': True},
 'Pset_EnvironmentalImpactIndicators': {'Reference': 'Перекрытие:КЖ0_Перекрытие',
  'id': 231},
 'Pset_ReinforcementBarPitchOfSlab': {'Reference': 'Перекрытие:КЖ0_Перекрытие',
  'id': 233},
 'ADCM': {'ADCM_RD': 'КЖ1',
  'ADCM_GESN': '06-01-041-01',
  'ADCM_Level': 'L01',
  'ADCM_Part': '1',
  'ADCM_Title': 'АИ-11 О/22',
  'id': 243}}